![banner.png](https://storage.googleapis.com/kaggle-competitions/kaggle/5407/media/housesbanner.png)

# Ames Housing Data
Estates Agent are looking for the way to accurately predict saleprice of housing in Ames. By using data science to predict the prices, we will be able to come up with more competitive prices. We are also looking for features that homeowners can look into to potentially improve the property's sales value.
## Problem Statement

Create a regression model where we are able to see which features affect the price of property at sales in Iowa Ames

## Executive Summary

The data set that we will be exploring is the Ames Iowa Housing Dataset till the year 2010. In the Dataset there is a total of 81 features and 2051 columns. We will explore the data to see the correlation the features have to SalePrice, and also to build a regression model to predict the SalePrice of housing.

The final Kaggle models using Lasso, Ridge Regression uses 120 features to do the prediction with a RMSE score of around 28,000. At the end of the project, we will use the top 50 coefficient features to do our recommendations to homeowners and how they can improve their property prices using those features.

### Contents:
- [6. Pre-Processing](#6.-Pre-Processing)


Links:
[Kaggle challenge link](https://www.kaggle.com/c/dsi-us-6-project-2-regression-challenge/data)

# 6. Pre Processing

In [1]:
#Imports:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
plt.style.use('ggplot')

In [2]:
# Importing cleaned dataset for Pre Processing
df = pd.read_csv("../datasets/train_EDA.csv", na_filter=False)
df_test = pd.read_csv('../datasets/test.csv')

df.shape, df_test.shape

((2018, 37), (879, 80))

In [3]:
df.head()

,Id,MS Zoning,Street,Land Contour,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Year Built,...,Overall Total Value,Exter Total Value,Total Flr SF,Fireplace Total Value,Total Bath,Has Basement,Has Garage,Has Porch,Has Pool,Is New
0,109,RL,Pave,Lvl,Sawyer,RRAe,Norm,1Fam,2Story,1976,...,14,7,7.299797,0,3.0,1,1,1,0,0
1,544,RL,Pave,Lvl,SawyerW,Norm,Norm,1Fam,2Story,1996,...,12,7,7.660585,3,4.0,1,1,1,0,0
2,153,RL,Pave,Lvl,NAmes,Norm,Norm,1Fam,1Story,1953,...,12,7,6.964136,0,2.0,1,1,1,0,0
3,318,RL,Pave,Lvl,Timber,Norm,Norm,1Fam,2Story,2006,...,10,6,7.275865,0,3.0,1,1,0,0,1
4,255,RL,Pave,Lvl,SawyerW,Norm,Norm,1Fam,1.5Fin,1900,...,14,6,7.276556,0,2.0,1,1,1,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2018 entries, 0 to 2017
Data columns (total 37 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Id                     2018 non-null   int64  
 1   MS Zoning              2018 non-null   object 
 2   Street                 2018 non-null   object 
 3   Land Contour           2018 non-null   object 
 4   Neighborhood           2018 non-null   object 
 5   Condition 1            2018 non-null   object 
 6   Condition 2            2018 non-null   object 
 7   Bldg Type              2018 non-null   object 
 8   House Style            2018 non-null   object 
 9   Year Built             2018 non-null   int64  
 10  Year Remod/Add         2018 non-null   int64  
 11  Roof Style             2018 non-null   object 
 12  Exterior 1st           2018 non-null   object 
 13  Exterior 2nd           2018 non-null   object 
 14  Mas Vnr Type           2018 non-null   object 
 15  Bsmt

## 6.1 Scaling of Data

- will scale numerical data types for model

In [5]:
num_data = df.select_dtypes(['int64', 'float64']).keys()
num_data = [x for x in num_data if ((x != 'SalePrice') & (x != 'Id'))]

nums = df[num_data]
ss = StandardScaler()
ss.fit(nums)
nums_scaled = ss.transform(nums)

nums_scaled.shape

(2018, 21)

In [6]:
nums_scaled_pd = pd.DataFrame(nums_scaled, columns = num_data) #create pd for combining later
nums_scaled_pd.head()

,Year Built,Year Remod/Add,Bsmt Qual,Central Air,1st Flr SF,Full Bath,Kitchen Qual,TotRms AbvGrd,Garage Finish,Garage Cars,...,Overall Total Value,Exter Total Value,Total Flr SF,Fireplace Total Value,Total Bath,Has Basement,Has Garage,Has Porch,Has Pool,Is New
0,0.142892,0.995556,-0.552436,0.268819,-1.311506,0.799945,0.750726,-0.267463,0.312864,0.308899,...,1.416512,0.757663,0.154064,-0.893370,0.634353,0.161021,0.237791,0.654345,-0.058999,-0.583072
1,0.809074,0.614543,0.580507,0.268819,-0.583048,0.799945,0.750726,1.071845,0.312864,0.308899,...,0.199861,0.757663,1.311132,0.452018,1.730351,0.161021,0.237791,0.654345,-0.058999,-0.583072
2,-0.623217,1.090809,-0.552436,0.268819,-0.120233,-1.057695,0.750726,-0.937117,-0.806568,-1.023062,...,0.199861,0.757663,-0.922422,-0.893370,-0.461644,0.161021,0.237791,0.654345,-0.058999,-0.583072
3,1.142165,1.090809,0.580507,0.268819,-1.229783,0.799945,-0.782640,0.402191,1.432296,0.308899,...,-1.016789,-0.715327,0.077311,-0.893370,0.634353,0.161021,0.237791,-1.528246,-0.058999,1.715053
4,-2.388599,0.424037,-1.685379,0.268819,-0.880397,0.799945,-0.782640,-0.267463,-0.806568,0.308899,...,1.416512,-0.715327,0.079529,-0.893370,-0.461644,0.161021,0.237791,0.654345,-0.058999,-0.583072


- Numerical features scaled!

## 6.2 One-hot encode categorical variables

- Creating dummies for model

In [7]:
#selecting object dtypes to create dummies
obj_data = df.select_dtypes(['object']).keys()
print(obj_data)
len(obj_data)

Index(['MS Zoning', 'Street', 'Land Contour', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Roof Style', 'Exterior 1st',
       'Exterior 2nd', 'Mas Vnr Type', 'Heating', 'Garage Type'],
      dtype='object')


14

In [8]:
obj_processed = pd.get_dummies(df[obj_data], columns = obj_data)
obj_processed.head()

,MS Zoning_C (all),MS Zoning_FV,MS Zoning_I (all),MS Zoning_RH,MS Zoning_RL,MS Zoning_RM,Street_Grvl,Street_Pave,Land Contour_Bnk,Land Contour_HLS,...,Heating_Grav,Heating_OthW,Heating_Wall,Garage Type_2Types,Garage Type_Attchd,Garage Type_Basment,Garage Type_BuiltIn,Garage Type_CarPort,Garage Type_Detchd,Garage Type_NA
0,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [9]:
#remove columns with NA count values
na_col = obj_processed.filter(regex = 'NA')
na_col_keys = na_col.keys()
na_col_keys

Index(['Neighborhood_NAmes', 'Mas Vnr Type_NA', 'Garage Type_NA'], dtype='object')

In [10]:
# dropping NA columns
obj_processed.drop(columns = na_col_keys, inplace = True)

In [11]:
df_new = pd.concat([df['Id'], nums_scaled_pd, obj_processed, df['SalePrice']], ignore_index=False, sort=False, axis = 1)
df_new.shape

(2018, 143)

In [12]:
df_new.columns

Index(['Id', 'Year Built', 'Year Remod/Add', 'Bsmt Qual', 'Central Air',
       '1st Flr SF', 'Full Bath', 'Kitchen Qual', 'TotRms AbvGrd',
       'Garage Finish',
       ...
       'Heating_Grav', 'Heating_OthW', 'Heating_Wall', 'Garage Type_2Types',
       'Garage Type_Attchd', 'Garage Type_Basment', 'Garage Type_BuiltIn',
       'Garage Type_CarPort', 'Garage Type_Detchd', 'SalePrice'],
      dtype='object', length=143)

## 6.2 Preprocess test data

- the same transformation to the Train data will be done on the test data for prediction later

In [13]:
df_test.shape

(879, 80)

### 6.2.1 Changing Categorical datas to ordinal

In [14]:
#Copied from Data cleaning to apply on Test data
labels = {'Exter Qual': {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1} }
df_test.replace(labels, inplace=True)


labels = {'Exter Cond': {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1} }
df_test.replace(labels, inplace=True)

labels = {'Bsmt Qual': {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0} }
df_test.replace(labels, inplace=True)


labels = {'Bsmt Cond': {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0} }
df_test.replace(labels, inplace=True)

labels = {'Bsmt Exposure': {'Gd': 4, 'Av': 3, 'Mn': 2, 'No': 1, 'NA': 0} }
df_test.replace(labels, inplace=True)


labels = {'BsmtFin Type 1': {'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf': 1, 'NA': 0} }
df_test.replace(labels, inplace=True)

labels = {'Heating QC': {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1} }
df_test.replace(labels, inplace=True)


labels = {'Central Air': {'Y': 1, 'N': 0} }
df_test.replace(labels, inplace=True)


labels = {'Kitchen Qual': {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1} }
df_test.replace(labels, inplace=True)

labels = {'Functional': {'Typ': 8, 'Min1': 7, 'Min2': 6, 'Mod': 5, 'Maj1': 4, 
                         'Maj2': 3, 'Sev': 2, 'Sal': 1} }
df_test.replace(labels, inplace=True)

labels = {'Fireplace Qu': {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0} }
df_test.replace(labels, inplace=True)


labels = {'Garage Finish': {'Fin': 3, 'RFn': 2, 'Unf': 1, 'NA': 0} }
df_test.replace(labels, inplace=True)


labels = {'Garage Qual': {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0} }
df_test.replace(labels, inplace=True)


labels = {'Garage Cond': {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0} }
df_test.replace(labels, inplace=True)


labels = {'Lot Shape': {'Reg': 4, 'IR1': 3, 'IR2': 2, 'IR3': 1} }
df_test.replace(labels, inplace=True)


labels = {'Utilities': {'AllPub': 3, 'NoSewr': 2, 'NoSeWa': 1} }
df_test.replace(labels, inplace=True)



labels = {'Land Slope': {'Gtl': 3, 'Mod': 2, 'Sev': 1} }
df_test.replace(labels, inplace=True)


labels = {'Foundation': {'BrkTil': 6, 'CBlock': 5, 'PConc': 4, 'Slab': 3, 'Stone': 2, 'Wood': 1} }
df_test.replace(labels, inplace=True)


labels = {'BsmtFin Type 2': {'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf': 1, 'NA': 0} }
df_test.replace(labels, inplace=True)


labels = {'Electrical': {'SBrkr': 5, 'FuseA': 4, 'FuseF': 3, 'FuseP': 2, 'Mix': 1, 'NA': 0} }
df_test.replace(labels, inplace=True)


labels = {'Paved Drive': {'Y': 2, 'P': 1, 'N': 0} }
df_test.replace(labels, inplace=True)


labels = {'Pool QC': {'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1, 'NA': 0} }
df_test.replace(labels, inplace=True)


labels = {'Fence': {'GdPrv': 4, 'MnPrv': 3, 'GdWo': 2, 'MnWw': 1, 'NA': 0} }
df_test.replace(labels, inplace=True)

### 6.2.2 Creating new features

In [15]:
#Copied from Data cleaning to apply on Test data

df_test['Bsmt Total Value'] = (df_test['Bsmt Qual'] + df_test['Bsmt Cond'] + df_test['Bsmt Exposure']+ 
                          df_test['BsmtFin Type 1']+ df_test['BsmtFin Type 2']+ df_test['Bsmt Full Bath'])


df_test['Garage Total Value'] = (df_test['Garage Finish'] + df_test['Garage Cars'] + df_test['Garage Qual']+ 
                          df_test['Garage Cond'])


df_test['Lot Total Value'] = ((df_test['Lot Frontage'] + df_test['Lot Area']))


df_test['Overall Total Value'] = (df_test['Overall Qual'] + df_test['Overall Cond'])


df_test['Exter Total Value'] = (df_test['Exter Qual'] + df_test['Exter Cond'])


df_test['Total Flr SF'] = (df_test['1st Flr SF'] + df_test['2nd Flr SF'])


df_test['Fireplace Total Value'] = (df_test['Fireplaces'] * df_test['Fireplace Qu'])


df_test['Total Porch Area'] = (df_test['Open Porch SF'] + df_test['Enclosed Porch'] +
                          df_test['3Ssn Porch'] + df_test['Screen Porch'])


df_test['Total Bath'] = (df_test['Bsmt Full Bath'] + df_test['Bsmt Half Bath'] +
                          df_test['Full Bath'] + df_test['Half Bath'])


df_test['Kitchen Total Value'] = (df_test['Kitchen AbvGr'] * df_test['Kitchen Qual'])


# adding in some boolean features
df_test['Has Basement'] = df_test['Total Bsmt SF'].apply(lambda x: 1 if x > 0 else 0)

df_test['Has Garage'] = df_test['Garage Area'].apply(lambda x: 1 if x > 0 else 0)

df_test['Has Porch'] = df_test['Total Porch Area'].apply(lambda x: 1 if x > 0 else 0)

df_test['Has Pool'] = df_test['Pool Area'].apply(lambda x: 1 if x > 0 else 0)

df_test['Was Remodeled'] = (df_test['Year Remod/Add'] != df_test['Year Built']).astype(np.int64)

df_test['Is New'] = (df_test['Year Built'] > 2000).astype(np.int64)

cat_num_features = ['Exter Qual','Exter Cond','Bsmt Qual','Bsmt Cond','Bsmt Exposure','BsmtFin Type 1','Heating QC',
                    'Kitchen Qual','Functional','Fireplace Qu','Garage Finish','Garage Qual','Garage Cond',
                   'Lot Shape','Utilities','Land Slope','Foundation','BsmtFin Type 2','Electrical','Paved Drive',
                   'Pool QC','Fence','MS Subclass', 'Overall Qual', 'Overall Cond','Bsmt Full Bath', 'Bsmt Half Bath',
                    'Full Bath', 'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'TotRms Abv Grd', 'Fireplaces', 
                    'Garage Yr Blt','Garage Cars', 'Misc Val', 'Mo Sold', 'Yr Sold', 'Bsmt Total Value', 
                    'Garage Total Value', 'Overall Total Value', 'Exter Total Value', 'Fireplace Total Value',
                    'Total Bath', 'Kitchen Total Value', 'TotRms AbvGrd']

boolean_data = ['Central Air', 'Has Basement', 'Has Garage', 'Has Porch',
                'Has Pool', 'Was Remodeled', 'Is New']


year_data = df_test.filter(regex = 'Yr').keys() | df_test.filter(regex = 'Year').keys()
date_data = year_data | df.filter(regex = 'Mo').keys() 

num_data = df_test.select_dtypes(['int64', 'float64']).keys()

num_no_id = [x for x in num_data if ((x != 'SalePrice') & (x != 'Id'))]

num_no_date = [f for f in num_no_id if f not in date_data]

# remove boolean features
num_no_bool = [f for f in num_no_date if f not in boolean_data]

# remove num categorical features
num_features = [f for f in num_no_bool if f not in cat_num_features] 

#log continuous data type
df_test[num_features] = np.log1p(df_test[num_features])


In [16]:
col_diff = df_test.columns.difference(df.columns)
df_test.drop(columns = col_diff, inplace = True)

###  6.2.3 Scaling Test Data

In [17]:
test_num_data = df_test.select_dtypes(['int64', 'float64']).keys()
test_num_data = [x for x in test_num_data if ((x != 'SalePrice') & (x != 'Id'))]

test_nums = df_test[test_num_data]

test_nums_scaled = ss.transform(test_nums)

test_nums_scaled.shape

(879, 21)

In [18]:
test_nums_scaled_pd = pd.DataFrame(test_nums_scaled, columns = test_num_data) #create pd for combining later
test_nums_scaled_pd.head()

,Year Built,Year Remod/Add,Bsmt Qual,Central Air,1st Flr SF,Full Bath,Kitchen Qual,TotRms AbvGrd,Garage Finish,Garage Cars,...,Overall Total Value,Exter Total Value,Total Flr SF,Fireplace Total Value,Total Bath,Has Basement,Has Garage,Has Porch,Has Pool,Is New
0,-2.055508,-1.623906,-1.685379,-3.719978,-0.600400,0.799945,-2.316005,1.741499,-0.806568,-1.023062,...,1.416512,-2.188317,1.003805,NaN,-0.461644,0.161021,0.237791,0.654345,-0.058999,-0.583072
1,0.176202,-0.337988,0.580507,0.268819,1.843061,0.799945,-0.782640,2.411153,1.432296,0.308899,...,-1.625115,-0.715327,1.067997,NaN,-0.461644,0.161021,0.237791,-1.528246,-0.058999,-0.583072
2,1.142165,1.043183,0.580507,0.268819,-1.589131,0.799945,0.750726,0.402191,0.312864,0.308899,...,0.199861,0.757663,0.190692,0.900481,1.730351,0.161021,0.237791,0.654345,-0.058999,1.715053
3,-1.622490,1.043183,-0.552436,0.268819,-0.398200,-1.057695,-0.782640,-0.937117,-0.806568,0.308899,...,-0.408464,0.757663,-1.204229,NaN,-1.557642,0.161021,0.237791,0.654345,-0.058999,-0.583072
4,-0.290126,-1.004761,0.580507,0.268819,0.754478,-1.057695,-0.782640,-0.267463,0.312864,0.308899,...,-0.408464,-0.715327,-0.035626,2.694332,0.634353,0.161021,0.237791,0.654345,-0.058999,-0.583072


### 6.2.4 get_dummies test Data

In [19]:
test_obj_data = df_test.select_dtypes(['object']).keys()
test_obj_processed = pd.get_dummies(df_test[test_obj_data], columns = test_obj_data)
test_obj_processed.head()

,MS Zoning_C (all),MS Zoning_FV,MS Zoning_I (all),MS Zoning_RH,MS Zoning_RL,MS Zoning_RM,Street_Grvl,Street_Pave,Land Contour_Bnk,Land Contour_HLS,...,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Garage Type_2Types,Garage Type_Attchd,Garage Type_Basment,Garage Type_BuiltIn,Garage Type_CarPort,Garage Type_Detchd
0,0,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1
1,0,0,0,0,1,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
2,0,0,0,0,1,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
3,0,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1
4,0,0,0,0,1,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0


### 6.2.5 Creating processed test dataset

In [20]:
df_test_new = pd.concat([df_test['Id'], test_nums_scaled_pd, test_obj_processed], ignore_index=False, sort=False, axis = 1)
df_test_new.shape

(879, 136)

In [21]:
df_new.shape

(2018, 143)

#### 6.2.5 comments 

Different number of columns between dataset, will probably do further adjustments before final prediction

## 6.3 Exporting both datasets for model

In [22]:
df_new.to_csv("../datasets/train_preprocess.csv", index=False)
df_test_new.to_csv("../datasets/test_preprocess.csv", index=False)

## continue to Model and Analysis